In [26]:
# Gerekli kütüphaneleri yükleyelim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

train_path = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/train.csv"

test_path = "C:/Users/ASUS/Desktop/Regression of Used Car Prices/test.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


# Eksik verileri doldurma
train_data['clean_title'].fillna('Unknown', inplace=True)
test_data['clean_title'].fillna('Unknown', inplace=True)

# Kategorik verileri one-hot encoding ile sayısal değerlere dönüştürme
categorical_columns = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_train_array = encoder.fit_transform(train_data[categorical_columns])
encoded_test_array = encoder.transform(test_data[categorical_columns])

# Encoder tarafından oluşturulan yeni sütun adlarını alalım
encoded_columns = encoder.get_feature_names_out(categorical_columns)

# Numpy array'lerini DataFrame'e çevirirken sütun adlarını ekleyelim
encoded_train_data = pd.DataFrame(encoded_train_array, columns=encoded_columns, index=train_data.index)
encoded_test_data = pd.DataFrame(encoded_test_array, columns=encoded_columns, index=test_data.index)

# Orijinal veriyle birleştirme
train_data = train_data.join(encoded_train_data).drop(columns=categorical_columns)
test_data = test_data.join(encoded_test_data).drop(columns=categorical_columns)

# Bağımlı ve bağımsız değişkenler
X = train_data.drop(columns=['price', 'id'])
y = train_data['price']

# Veri setini eğitim ve doğrulama setlerine ayırma
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost modelini tanımlama
catboost_model = CatBoostRegressor(
    iterations=1000,            # Model 1000 ağaçta eğitilecek
    learning_rate=0.05,         # Her iterasyonda ağırlıklar 0.05 oranında güncellenecek
    depth=8,                    # Karar ağaçları maksimum 8 derinlikte olacak
    loss_function='RMSE',       # Kayıp fonksiyonu olarak RMSE kullanılacak
    eval_metric='RMSE',         # Performans metriği olarak RMSE kullanılacak
    random_state=42,            # Eğitim sürecinin tekrarlanabilir olması için rastgelelik tohumunu ayarlayacak
    verbose=100,                # Her 100 iterasyonda modelin performansının çıktısını verecek
    early_stopping_rounds=50    # Doğrulama setinde performans iyileşmediğinde 50 iterasyon sonra eğitim duracak
)


# Modeli eğitme
catboost_model.fit(
    X_train, y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100,  # Erken durdurma için
)

# Tahminleri yapma
y_pred = catboost_model.predict(X_valid)

# Performans değerlendirmesi (RMSE)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"Validation RMSE: {rmse}")


0:	learn: 79401.0355289	test: 74093.9229627	best: 74093.9229627 (0)	total: 67.9ms	remaining: 1m 7s
100:	learn: 72439.8853700	test: 69110.6451805	best: 69110.6451805 (100)	total: 3.52s	remaining: 31.3s
200:	learn: 70781.9495748	test: 69134.6881001	best: 69104.0710772 (125)	total: 6.99s	remaining: 27.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 69104.07108
bestIteration = 125

Shrink model to first 126 iterations.
Validation RMSE: 69104.07107724219


In [28]:
# 6. Batch'ler Halinde Test Setinde Tahminler
batch_size = 1000
all_predictions = pd.DataFrame()

for i in range(0, len(test_data), batch_size):
    batch = test_data[i:i+batch_size]
    batch_predictions = catboost_model.predict(batch.drop(columns=['id']))
    
    batch_output = pd.DataFrame({'id': batch['id'], 'price': batch_predictions})
    all_predictions = pd.concat([all_predictions, batch_output])

# 7. Sonuçları CSV Dosyasına Kaydetme
all_predictions.to_csv('predictions.csv', index=False)